In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.5MB/s 
     |████████████████████████████████| 2.9MB 14.6MB/s 
     |████████████████████████████████| 890kB 44.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=10e1087a6d0028ffb81391b1787b5cc24eef67c91f47886c91d34ef102fa11b2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6MB 1.2MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338489 sha256=9949bc0fce6e2963dd64f6d5876afd3b111deb756a5200de253cb8db5d5e78a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-9pettdh_/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import FlaubertModel, FlaubertTokenizer
from scipy.spatial.distance import cosine
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
import fr_core_news_md

class Summarizer():

  def __init__(self, stop_words=None):
    self.nlp = fr_core_news_md.load()
    self.stop_words = stop_words

###### Load Model Methods ######

  def init_model(self, model='flaubert', device=None, log=False):
    # Choosing device for language model
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.device = device

    try:
      # Flaubert model
      if model=='flaubert':
        model_name = 'flaubert/flaubert_large_cased'
        flaubert = FlaubertModel.from_pretrained(model_name)
        tokenizer = FlaubertTokenizer.from_pretrained(model_name)
        self.model = flaubert
        self.tokenizer = tokenizer
        self.model_name = model_name
      # Camembert model
      elif model=='camembert':
        model_name = 'camembert'
        self.model = torch.hub.load('pytorch/fairseq', 'camembert')
        self.model_name = model_name
    except:
        print(f'Error while loading the {model} model.')
        return

    # Model Inference
    self.model.to(self.device)
    self.model.eval()

    # Log Info
    if log:
      self.init_log(self.model_name, self.device)

  def init_log(self, model_name, device):
    print(f'Summarizer: \'{model_name}\' successfully loaded on {device}.')

  def to(self, device):
    """
    Moves and/or casts the NLP model parameters and buffers.
    Parameters
    ----------
      device: string | device name.
    """

    self.device = device
    self.model.to(device)

###### Sentence Selection Methods ######

  def reference_selection(self, reference_embeddings, embeddings, threshold):
    """
    Filter embeddings based on similarity with a reference embedding.
    Return selected embeddings with similarity score higher than thresold.
    """

    selected_indices = []
    for reference_embedding in reference_embeddings:
      similarities = np.array(self.get_similarities(reference_embedding,
                                                    embeddings))
      
      filtered_indices = np.where(similarities > threshold)[0]
      selected_indices.extend(filtered_indices.tolist())

    return sorted(list(set(selected_indices)))

###### Keyword Selection Methods #####

  def word_embedding(self, word):
    """
    Return model embedding of the given word.
    """
    # Camembert
    if self.model_name=='camembert':
      token = self.model.encode(word).to(self.device)

      with torch.no_grad():
        encoded_layers = self.model.extract_features(token,
                                                     return_all_hiddens=False)
        embedded_word = encoded_layers[0][0].cpu().numpy()

      return embedded_word
    # Flaubert
    else:
      token_ids = torch.tensor([self.tokenizer.encode(word,
                                                      add_special_tokens=False)])
      token_ids = token_ids.to(self.device)
    
      with torch.no_grad():
        last_layers = self.model(token_ids)

      token_embedding = torch.stack(last_layers, dim=0)[0]
      word_embedding = torch.mean(token_embedding,dim=1)
      embedded_word = word_embedding.cpu().numpy()
      
      return embedded_word

  def remove_stop_words(self, sentence):
    """
    Remove stop words form a text sentence.
    """
    split = [word for word in sentence.split(' ') if len(word) > 2]
    sentence = ' '.join(split)
    sentence = self.nlp(sentence)
    tokens = [token.text for token in sentence]
    clean_sentence = tokens
    if self.stop_words is not None:
      clean_sentence = [word for word in tokens if not word in self.stop_words]
    clean_sentence[:] = [item for item in clean_sentence if item != ' ']

    return clean_sentence

  def content_words_embedding(self, text):
    """
    Return the word granularity text embedding of the given text.
    """

    text_content_words = [self.remove_stop_words(sentence) for sentence in text]

    content_words_embedding = []
    for words in text_content_words:
      content_words_embedding.append([self.word_embedding(word) \
                                      for word in words])
    
    return content_words_embedding

  def keyword_similarity(self, content_words_embedding, keyword_embedding):
    
    keyword_similarities = []
    for words_embedding in content_words_embedding:
      if len(words_embedding) != 0:
        sim = [1 - cosine(keyword_embedding, w) for w in words_embedding]
      else:
        sim = [0.]
      keyword_similarities.append(sim)
    
    return keyword_similarities

  def keyword_selection(self, content_words_embedding, keywords_embeddings,
                        method='max', threshold=0.6):
    """
    Return selected text indices based on max/mean similarity with keywords.
    """
    kw_similarities = [self.keyword_similarity(content_words_embedding,
                                               kw) for kw in keywords_embeddings]
    
    top_indices = []
    for kw_similarity in kw_similarities:

      top_sim = []
      if method=='max':
        max_sim_sentence = [max(sentence) for sentence in kw_similarity]
        max_sim_sentence = np.array(max_sim_sentence)
        top_sim = np.where(max_sim_sentence >= threshold)[0]
      else:
        mean_sim_sentence = [np.mean(sentence, axis=0) for sentence in kw_similarity]
        mean_sim_sentence = np.array(mean_sim_sentence)
        top_sim = np.where(mean_sim_sentence >= threshold)[0].tolist()

      top_indices.extend(top_sim)

    return list(set(top_indices))

###### "FIT" methods ######

  def fit(self, text,
          reference_sentences=None,
          reference_threshold=0.6,
          keywords=None,
          keywords_method='max',
          keywords_threshold=0.6,
          log=True):
    # Embed all the text
    try:
      if not isinstance(text, pd.core.series.Series):
        text = pd.Series(text)
    except:
      print('Data input error: text should be a numpy ndarray or a pandas '
            'series of str sentences')
      return
    
    self.text = text.to_numpy()
    if self.model_name == 'camembert':
      self.text_embeddings = self.camembert_text_embedding(self.text)
    else:
      self.text_embeddings = self.flaubert_text_embedding(self.text)

    # Reference Sentence Selection
    if reference_sentences is not None:
      if self.model_name == 'camembert':
        reference_embeddings = self.camembert_text_embedding(reference_sentences)
      else:
        reference_embeddings = self.flaubert_text_embedding(reference_sentences)

      selected_indices = self.reference_selection(reference_embeddings,
                                                  self.text_embeddings,
                                                  reference_threshold)
      self.text = self.text[selected_indices]
      self.text_embeddings = self.text_embeddings[selected_indices]

    # Keyword Sentence Selection
    if keywords is not None:
      keywords_embeddings = [self.word_embedding(keyword) for keyword in keywords]

      content_words_embedding = self.content_words_embedding(self.text)
      selected_indices = self.keyword_selection(content_words_embedding,
                                                keywords_embeddings,
                                                method=keywords_method,
                                                threshold=keywords_threshold)
      self.text = self.text[selected_indices]
      self.text_embeddings = self.text_embeddings[selected_indices]

    # Log Info
    if log:
      print(f'Summarizer fit: computed {self.text_embeddings.shape[0]} '
            f'embeddings of dim {self.text_embeddings.shape[1]}.')





  def flaubert_text_embedding(self, text):
    
    input_ids = [self.tokenizer.encode(sentence) for sentence in text]
    padded = np.array([i + [0]*(300-len(i)) for i in input_ids])

    attention_mask = np.where(padded != 0, 1, 0)
    input_ids_tensor = torch.tensor(padded).to(self.device)
    masks_tensor = torch.tensor(attention_mask).to(self.device)

    # with torch.no_grad():
    #   encoded_layers = self.model(input_ids_tensor, masks_tensor)


    # token_embeddings = torch.stack(encoded_layers, dim=0)[0]
    # sentence_embedding = torch.mean(token_embeddings,dim=1)
    embedded_sentences =masks_tensor.cpu().numpy()

    return embedded_sentences

###### "Summary" methods ######

  def get_similarities(self, reference_embedding, embeddings):
    """
    Return the similarity scores between reference and embeddings.
    """
    similarities = []
    for i in range(len(embeddings)):
        sim = 1 - cosine(reference_embedding, embeddings[i])
        similarities.append(sim)
    
    return similarities

  def top_similarities(self, reference_embedding, embeddings, nb_top):
    """
    Return the nb_top embeddings indices closer to the reference_embedding.
    Parameter
    ---------
      reference_embedding: np.array | reference embedding for distance.
      embeddings: np.ndarray | embeddings to sort according to ref distance.
      nb_top: int | number of top indices to return.
    """

    # Compute similarity according to distance to reference.
    similarities = self.get_similarities(reference_embedding, embeddings)

    # Return nb_top indices
    top_indices = np.array(similarities).argsort()[::-1][:nb_top]
    return top_indices

  def mean_similarity_summary(self, nb_sentences=5, return_indices=False):
    """
    Perform summarization over the text_embeddings with mean similarity method.
    The mean embedding is used as reference for similarity.
    Return the summary of length nb_sentences.
    (optional) Return nb_sentences indices ordered by distance to the mean.
    Parameters
    ----------
      nb_sentences: int | length of the summary.
      return_indices: bool | return sentences indices if set to True.
    """
    # Compute mean sentence embedding
    mean_sentence_embedding = np.mean(self.text_embeddings, axis=0)

    top_indices = self.top_similarities(mean_sentence_embedding,
                                        self.text_embeddings,
                                        nb_sentences)
    
    summary = self.text[sorted(top_indices)]

    if return_indices:
      return summary, top_indices

    return summary

In [ ]:
import numpy as np
def summarize(model, method, text, nb_sentences):
    summarizer = Summarizer()
    summarizer.init_model(model, log=True)        
    summarizer.fit(text)
    if method == 'mean':
        summary = summarizer.mean_similarity_summary(nb_sentences=nb_sentences)
    return summary

text="""La Terre est la troisième planète par ordre d'éloignement au Soleil et la cinquième plus grande aussi bien par la masse que le diamètre du Système solaire. Par ailleurs, elle est le seul objet céleste connu pour abriter la vie. Elle orbite autour du Soleil en 365,256 jours solaires — une année sidérale — et réalise une rotation sur elle-même relativement au Soleil en 23 h 56 min 4 s — un jour sidéral — soit un peu moins que son jour solaire de 24 h du fait de ce déplacement autour du Soleil. L'axe de rotation de la Terre possède une inclinaison de 23°, ce qui cause l'apparition des saisons.
D'après la datation radiométrique, la Terre s'est formée il y a 4,54 milliards d'années. Elle possède un unique satellite naturel, la Lune, qui s'est formée peu après. L'interaction gravitationnelle avec son satellite crée les marées, stabilise son axe de rotation et réduit graduellement sa vitesse de rotation. La vie serait apparue dans les océans il y a au moins 3,5 milliards d'années, ce qui a affecté l'atmosphère et la surface terrestre par la prolifération d'organismes d'abord anaérobie puis, suite à l'explosion cambrienne, aérobie. Une combinaison de facteurs tels que la distance de la Terre au Soleil environ 150 millions de kilomètres, aussi appelée unité astronomique, son atmosphère, sa couche d'ozone, son champ magnétique et son évolution géologique ont permis à la vie d'évoluer et de se développer. Durant l'histoire évolutive du vivant, la biodiversité a connu de longues périodes d'expansion occasionnellement ponctuées par des extinctions massives ; environ 99 % des espèces qui ont un jour vécu sur Terre sont maintenant éteintes. En 2020, plus de 7,7 milliards d'êtres humains — répartis en environ 200 États — vivent sur Terre et dépendent de sa biosphère et de ses ressources naturelles pour leur survie. Il est estimé que la Terre devrait pouvoir maintenir la vie telle que connue actuellement durant encore au moins 500 millions d'années.
Elle est la planète la plus dense du Système solaire ainsi que la plus grande et massive des quatre planètes telluriques. Son enveloppe rigide — appelée la lithosphère — est divisée en différentes plaques tectoniques qui migrent de quelques centimètres par an. Environ 71 % de la surface de la planète est couverte d'eau — notamment des océans, mais aussi des lacs et rivières, constituant l'hydrosphère — et les 29 % restants sont des continents et des îles. La majeure partie des régions polaires est couverte de glace, notamment avec l'inlandsis de l'Antarctique et la banquise de l'océan Arctique. La structure interne de la Terre est géologiquement active, le noyau interne solide et le noyau externe liquide composés tous deux essentiellement de fer permettant notamment de générer le champ magnétique terrestre par effet dynamo et la convection du manteau terrestre (composé de roches silicatées) étant la cause de la tectonique des plaques."""

def load_preprocess_text(text):  
    text = text.split('.')
    return np.array(text)

text = load_preprocess_text(text)
summary = summarize('flaubert','mean',text,5)
for sentence in summary:
    print(sentence)



Summarizer: 'flaubert/flaubert_large_cased' successfully loaded on cpu.
Summarizer fit: computed 18 embeddings of dim 300.
La Terre est la troisième planète par ordre d'éloignement au Soleil et la cinquième plus grande aussi bien par la masse que le diamètre du Système solaire
 L'interaction gravitationnelle avec son satellite crée les marées, stabilise son axe de rotation et réduit graduellement sa vitesse de rotation
 En 2020, plus de 7,7 milliards d'êtres humains — répartis en environ 200 États — vivent sur Terre et dépendent de sa biosphère et de ses ressources naturelles pour leur survie
 Environ 71 % de la surface de la planète est couverte d'eau — notamment des océans, mais aussi des lacs et rivières, constituant l'hydrosphère — et les 29 % restants sont des continents et des îles
 La majeure partie des régions polaires est couverte de glace, notamment avec l'inlandsis de l'Antarctique et la banquise de l'océan Arctique
